In [1]:
#Edited: 2025: Sabrina
#Edited 2020: Johannes
# This is an example script for the use of the PointConcentration class/object for pointconcentration measurements
#from the windtunnel python package, showing the use of almost all available analysis functions.

# The columns in the input file are expected to be time, wtref, slow FID, fast ID, release signal and open_rate, 
# where release signal will be ignored.
# The PointConcentration class returns a DataFrame, using the standard measurement txt-file output as input

In [2]:
#Imports
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#Path and file names

# Add the parent directory to the path for the windtunnel package import
path_dir = "/home/sabrina/Desktop/Schreibtisch/Arbeit_2025/WTSoftwareUtilitiesShare"
sys.path.append(os.path.abspath(path_dir))
import windtunnel as wt

#Path to your input data
path = f"{path_dir}/ExampleData/InputData/Zeitserie1/"
# Name of your measurement files prefix
namelist = ['BFS_BD3_MP01_000']

#Path to your output folder for average files and plots
output_path = f"{path_dir}/ExampleData/Results/"

#For PointData for the functions to work the columns of the file should be: time, wtref, slow_FID, fast_FID, open_rate
#(See manual for the description of the variables)

#Name of csv file which contains ambient conditions data. Multiple diff. ambient conditions for diff datasets can be read-in at ones
#If no file given or configuration wrong, the program ressorts to try reading-in given values manually below. 
#csv_file='ambient_conditions.csv'
csv_file= f"{path_dir}/ExampleData/ParameterFiles/ambient_conditions.csv"

In [4]:
#Variables and Parameters set for all ts, if no ambient_conditions.csv file overgiven

#If at the end calculate entdimensionalised or full scale transform quantities
#Default: nd:entdimensionalise, ms:model scale, fs:full scale.    
full_scale='ms'

#Postprocessing before analysis
applyPostprocessing=True
averageInterval=60 #s  #Interval to downaverage raw time series to before analysis

#Overgive uncertainty
uncertainty_value=None #Uncertainty of concentration, has to be calculated/estimated from the experimentator
#If None overgive no error visualized as errorbars 0.5
uncertainty_representation="percentage" #"absoluteValue"

In [5]:
#Example file/Default environment values if no csv_file found:

#Source location  [mm]
x_source=0
y_source=0
z_source=0

#Source mass flow controller, calibration settings
mass_flow_controller=0.300 #[l/h]*1/100 #'X'  #Controller(settings) used, just a name placeholder for orientation, not used yet
#If calibration performed on a controller, corrects actual max. flow capacity of controller
calibration_curve=0.3     #0.3 oder 3
calibration_factor=0 #1      #

#Gas characteristics
gas_name='C12'           #Just placeholder name variable for orientation, not used for anything
gas_factor=0.5   #[-]    #!!! Needs to be calculate/specificate f.e. if gas changes 
mol_weight=29.0 #28.97 #Air [g/mol]


#Measurement location [mm]
x_measure=1020 #855.16
y_measure= 0    #176.29
z_measure= 5     #162

#Surrounding conditions
pressure=101426.04472        #1009.38  #[hPa] ->Pa
temperature=23             #23.5  #[°C]

#Model to Reality scaling
scale=400                     #250      #Model/Reality
scaling_factor=0.5614882               #0.637       #USA1 to selected ref pos.?
ref_length=1/400              #1/250           #Lref
ref_height=100/400            #None            #Href


full_scale_wtref=10             #6         #Uref_fullscale
full_scale_flow_rate=0.002     #Q_amb[kg/s]?   #0.5   #Qv_fullscale
full_scale_temp=20             #[°C]
full_scale_pressure=101325     #[Pa]
#Q_ambient[kg/s] ->  Q[m³/s]=Q[kg/s]*R*T/(M*p)

#Variable wdir for wind direction. To be implemented in future. ##
#wdir=0
#Variable axis_range. Reserved for future implementation of axis range specification, 
#analogously to puff mode
#axis_range='auto'

In [6]:
###### Initialise concentration ts dictionary of length namelist, as well as for full scale and entdimensionalised
conc_ts = {}
conc_ts.fromkeys(namelist)
conc_ts_fs = conc_ts
conc_ts_nd = conc_ts

dict_conc_ts = conc_ts
dict_conc_nd = conc_ts
dict_conc_fs = conc_ts

data_dict = {}
data_dict.fromkeys(namelist)

{'BFS_BD3_MP01_000': None}

In [7]:
#Testing nothing should be read-in already
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].x_source)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].temperature)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].mass_flow_controller)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].scaling_factor)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].gas_factor)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].full_scale_flow_rate)

StopIteration: 

In [ ]:
#Read in ambient conditions for each folder or concentration ts from given csv file or for same conditions from manually
parameters_PerFolder = False  #True=for each folder/namelist entry new column, False: for each ts one column entry

for name in namelist:
    if parameters_PerFolder==True:
        #Read ambient conditions from csv file only for each folder
        ambient_conditions = wt.PointConcentration.get_ambient_conditions(path=path, name=name, input_file=csv_file)
        #Else read/use given default from cell above
        if ambient_conditions is None:
            []
        #Read ambient conditions from csv file
        else:
            x_source, y_source, z_source, x_measure, y_measure, z_measure, pressure, temperature, calibration_curve, mass_flow_controller, calibration_factor, scaling_factor, scale, ref_length, \
            ref_height, gas_name, mol_weight, gas_factor, full_scale_wtref, full_scale_flow_rate, full_scale_temp, full_scale_pressure = wt.PointConcentration.read_ambient_conditions(
                ambient_conditions, name)
        
    files = wt.get_files(path, name)
    print(files)

    #Initilise Dictionary for each given name containing dimensions of nr of files ts#0-
    conc_ts[name] = {}
    conc_ts[name].fromkeys(files)
    for file in files:
        if parameters_PerFolder == False:
            #Read in ambient condition column for each ts
            ambient_conditions = wt.PointConcentration.get_ambient_conditions(path=path, name=file, input_file=csv_file)
            #Else read/use given default from cell above
            if ambient_conditions is None:
                []
            #Read ambient conditions from csv file
            else:
                x_source, y_source, z_source, x_measure, y_measure, z_measure, pressure, temperature, calibration_curve, mass_flow_controller, calibration_factor, scaling_factor, scale, ref_length, \
                ref_height, gas_name, mol_weight, gas_factor, full_scale_wtref, full_scale_flow_rate, full_scale_temp, full_scale_pressure = wt.PointConcentration.read_ambient_conditions(
                ambient_conditions, file)
        
        conc_ts[name][file] = wt.PointConcentration.from_file(path + file)
    
        conc_ts[name][file].ambient_conditions(x_source=x_source, y_source=y_source, z_source=z_source,
                                               x_measure=x_measure, y_measure=y_measure, z_measure=z_measure,
                                               pressure=pressure,
                                               temperature=temperature,
                                               calibration_curve=calibration_curve,
                                               mass_flow_controller=mass_flow_controller,
                                               calibration_factor=calibration_factor)

#wdir,0,5

In [ ]:
#Set read-in scaling, gas and full scale information to internal class variables
for name in namelist:
    for file in files:

        conc_ts[name][file].scaling_information(scaling_factor=scaling_factor, 
                                                scale=scale,
                                                ref_length=ref_length, 
                                                ref_height=ref_height)
        
        conc_ts[name][file].tracer_information(gas_name=gas_name,
                                               mol_weight=mol_weight,
                                               gas_factor=gas_factor)
        
        conc_ts[name][file].full_scale_information(full_scale_wtref=full_scale_wtref,
                                                   full_scale_flow_rate=full_scale_flow_rate,
                                                   full_scale_temp=full_scale_temp,full_scale_pressure=full_scale_pressure)

In [ ]:
#Testing everything from ambient-conditions file now read-in
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].x_source)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].temperature)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].mass_flow_controller)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].scaling_factor)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].gas_factor)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].full_scale_flow_rate)
#self = conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))]

In [ ]:
#Calculate mass flow rate, net concentration and dimensionalise concentration
for name in namelist:
    for file in files:
        
        conc_ts[name][file].convert_temperature()
        conc_ts[name][file].calc_wtref_mean()
        
        conc_ts[name][file].calc_model_mass_flow_rate(usingMaxFlowRate="True",applyCalibration="False")
        conc_ts[name][file].calc_net_concentration()

        #conc_ts[name][file].clear_zeros()  #Remove values net_concentration =< 0 from dataset !noise
        conc_ts[name][file].calc_c_star()

        conc_ts[name][file].calc_full_scale_concentration() #Try

In [ ]:
#Test c*star output
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].c_star)
#Test full scale trafo
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration)
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].full_scale_concentration)

In [ ]:
plt.plot(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration)

In [ ]:
#Options for Outputting data in full-scale, model scale, and non-dimensionally.

for name in namelist:
    for file in files:
        
        if full_scale == 'ms':
            dict_conc_ts = conc_ts
            
        elif full_scale == 'fs':
            dict_conc_ts = conc_ts_fs
            dict_conc_ts[name][file].to_full_scale()
            
        elif full_scale == 'nd':
            dict_conc_ts = conc_ts_nd
            dict_conc_ts[name][file].to_non_dimensional()
        else:
            print(
                "Error: invalid input for full_scale. Data can only be computed in model scale (full_scale='ms'), full scale (full_scale='fs'), or non-dimensionally (full_scale=nd).")
        

In [ ]:
#Postprocessing if overgiven
measurementFreq=0.005 #Time series frequency #For now only for static case implemented
averageInterval=averageInterval #60 #s
columns=["net_concentration"] #Columns to average down

print(len(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))]))
#print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration)
if(applyPostprocessing==True):
    for name in namelist:
        for file in files:
            dict_conc_ts[name][file].downAverage(averageInterval=averageInterval,measurementFreq=measurementFreq, columns=columns)
            #dict_conc_ts[name][file].net_concentration

print(len(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration))

In [ ]:
print(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration)

In [ ]:
#Saving PointConcObject calculates new quantities(f.e. c*star) to files
#At the moment path hardcoded to windwos, but still ok
#default
os = "Windows"

if os=="Windows":
    folder = 'Point_Data\\' + name[:name.find('.')] + '\\'
elif os=="Linux":
     folder = 'Files/' + 'Point_Data/' + name[:name.find('.')] + '/'

for name in namelist:
    for file in files:
        
        wt.check_directory(output_path + folder)
        dict_conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].__check_sum = 8
        #dict_conc_ts[name][file].__check_sum = 8
       
        if full_scale == 'ms':
            dict_conc_ts[name][file].save2file_ms(file, out_dir=output_path + folder)
        elif full_scale == 'fs':
            dict_conc_ts[name][file].save2file_fs(file, out_dir=output_path + folder)
        elif full_scale == 'nd':
            dict_conc_ts[name][file].save2file_nd(file, out_dir=output_path + folder)
        else:
            print(
                "Error: invalid input for full_scale. Data can only be computed in model scale (full_scale='ms'), full scale (full_scale='fs'), or non-dimensionally (full_scale=nd).")

In [ ]:
#Saving averages and stats to files under folder Point_Data_avg/Point_Data_stats
#At the moment path hardcoded to windows, but still ok
os = "Windows"
if os=="Windows":
    folder_avg = 'Point_Data_avg\\' + name[:name.find('.')] + '\\'
    folder_stats = 'Point_Data_stats\\' + name[:name.find('.')] + '\\'
elif os=="Linux":
    folder_avg = 'Files/' + 'Point_Data_avg/' + name[:name.find('.')] + '/'
    folder_stats = 'Files/' + 'Point_Data_stats/' + name[:name.find('.')] + '/'

#Stats Full ausgabe: Mean, percentile 95, percentile 5, peak2Mean 
for name in namelist:
    for file in files: 
        wt.check_directory(output_path + folder_avg)
        wt.check_directory(output_path + folder_stats)
        dict_conc_ts[name][file].save2file_avg(file, out_dir=output_path + folder_avg)
        dict_conc_ts[name][file].save2file_fullStats(file, out_dir=output_path + folder_stats)  

In [ ]:
print(conc_ts[name][file].calc_percentiles(percentiles=[10, 90, 95], var='net_concentration'))
print(np.mean(conc_ts[name][file].net_concentration))
print(np.std(conc_ts[name][file].net_concentration))

In [ ]:
#Concentration fluctuation analysis 
#Intermittency based on threshold, peak2Mean, concentration variance spectral density distribution

#Seettings intermittency calculation
threshold_type="ratio" #ratio, absolute
threshold_method="mean" #mean, std
intermittency_threshold=1.5 #-> if type=ratio,method mean, threshold=threshold*mean(concentration), if type=absolute: threshold=threshold

conc_ts[name][file].analyze_concentration_fluctuations(dimensionless="False",
                                                       intermittency_threshold=intermittency_threshold,threshold_method=threshold_method)
#power(variance) of concentration changes for different frequencies/timer interval lengths 
#Low-frequency peak: Slow, gradual concentration changes
#High-frequency peak: Rapid, quick concentration fluctuations
#Broad spectrum: Mixed or complex concentration dynamics

In [ ]:
#Testing incorporation of STJF for intermittency (Sequential time join fourier, leaves time domain intact)
#STFTs calculates sequential FFTs and can be used as a way of quantifying the change of a spectrum over time.
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import stft

#0.005 6.0045 2.276105 12.835903 2.381506 3.91625 
#0.015 5.992015 1.911356 28.29813 2.390326 3.914681 
sampling_rate = 100#Hz

f, t, Zxx = stft(conc_ts[name][file].wtref, fs=sampling_rate, nperseg=256, noverlap=128)

plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
#plt.yscale('log')  #logarithmic scale?
plt.ylim(0,1)
#plt.colorbar(label='Magnitude')
plt.show()

In [ ]:
#For completeness also calculate further characteristic metrics of flow/ wind velocity time series (from wtref ts), skewness ..
#print(conc_ts[name][file].calculate_turbulence_intensity(dimensionless="True",returnDistribution="False",returnMetrics="True"))

In [ ]:
from windtunnel.concentration.CompareDatasets import compare_point_concentrations_3
#Give overview/comparison for the data, one plot including of of the plots choosen in the list "functionsForOverview
#Means, Pdf, Cdf, PowerDensity: Number of ts does not matter that much
#Scatterplot: use only 2 ts
#Histogram, Boxplot: would also recommend not to many, because of overlapping of the histograms for comparison, space for the boxplots..

#functionsForOverview = ["all"] #defaul -> all of the available plots
#all_plot_types = [
#        "Histogram", "Pdf", "Cdf", "Means", "BoxPlot", 
#        "QuantilPlot", "ScatterPlot", "ResidualPlot", "Autocorrelation"
#    ]
    
functionsForOverview = [
    "Histogram",
    "BoxPlot"
    #"",
    "Pdf",
    "Cdf",
    "Means",
    "PowerDensity"
        ]

#Choose which concentration time series/PointObjekts to show in the overview/comparison plot
DataPointsConc = [
    conc_ts[namelist[0]][files[0]],#
    conc_ts[namelist[0]][files[1]]
]

compare_point_concentrations_3(DataPointsConc,functionsForOverview)

In [ ]:
#Start visualisation single plots, adjustable
#ReadIn/Choose data (PointObjects) from the analysis above, to use for the separat plot visualisations, import plotting functions

#Specify files
#DataPointsConc = [
#    conc_ts[namelist[0]][files[0]],
#    conc_ts[namelist[0]][files[1]]
#]
#All files read-in
DataPointsConc=[]
for i in range(0,len(files)):
    DataPointsConc.append(
        conc_ts[namelist[0]][files[i]]
    )
#Labels    
labels = [f"Dataset {i}" for i in range(1,len(DataPointsConc)+1)]

print(labels)
print(DataPointsConc)

#Load all functions for plotting
from windtunnel.concentration.CompareDatasets import *

In [ ]:
xLabel="Datasets"
yLabel="Mean concentration[ppmV]" 
dimensionless="False"
xAchse = None 
yAchse=(72,79) #None 
error_values=0.2 #[0.5,0.2,0.1] #For error values overgive one number which is cast to all values, or an array if specify different errors for each measurements
errorType="absolute"
test = create_means(DataPointsConc,error_values,dimensionless=dimensionless,labels=None,xLabel=xLabel,yLabel=yLabel,xAchse=xAchse,yAchse=yAchse)
#plt.savefig("Mean_comparison.png",test) #To save image

In [ ]:
xLabel="Concentration[-]"
yLabel="Density"
dimensionless="True"
create_pdf(DataPointsConc,dimensionless="True",labels=None,xLabel=xLabel,yLabel=yLabel,xAchse=None,yAchse=None)

In [ ]:
xLabel="Datasets"
yLabel="Concentration[ppmV]"
xLabel="Datasets"
yLabel="Concentration[ppmV]"
create_histogram(DataPointsConc,dimensionless="False",labels=None,xLabel=xLabel,yLabel=yLabel,xAchse=None,yAchse=None)

"""
#import scipy.stats as stats
#import seaborn as sns
sns.histplot(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration, kde=True)
sns.displot(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration,kind="kde")
#mu = np.mean(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration)
#sigma = np.sqrt(np.var(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration))
#value = np.random.normal(loc=mu,scale=sigma,size=len(conc_ts[next(iter(conc_ts))][next(iter(conc_ts[next(iter(conc_ts))]))].net_concentration))
#sns.distplot(value)
"""

In [ ]:
yLabel=None
xLabel="Concentration[-]"
dimensionless="True"
create_cdf(DataPointsConc,dimensionless=dimensionless,labels=None,xLabel=xLabel,yLabel=yLabel,xAchse=None,yAchse=None)

In [ ]:
test = powerDensityPlot(DataPointsConc,dimensionless="False",plot=True,labels=None,xLabel=None,yLabel=None,xAchse=None,yAchse=None)
plt.savefig("test.png",test)

In [ ]:
#Example of how to load/Read-in averages from file to use f.e. in map or for plotting
from windtunnel.concentration.utils import load_avg_file

file_path = "/home/sabrina/Desktop/Schreibtisch/Arbeit_2025/windtunnel_software/Data/"
file_name = "Point_Data_avg\BFS_BD3_MP01_000_01\_avg_BFS_BD3_MP01_000_01.ts#0" #Location of saved file
data_dict = load_avg_file(file_path + file_name)


print(data_dict.keys())
print(data_dict['metadata'].keys())
print(data_dict['metadata']['x (measurement relativ to source)']['value'])
print(data_dict['metadata']['y (measurement relativ to source)']['value'])
print(data_dict['metadata']['z (measurement relativ to source)']['value'])

x = data_dict['metadata']['x (measurement relativ to source)']['value']
y = data_dict['metadata']['y (measurement relativ to source)']['value']
z = data_dict['metadata']['z (measurement relativ to source)']['value']
c = data_dict['data'][0]['net_concentration [ppmV]']

In [ ]:
pip install numpy_stl

In [ ]:
#Start visualisation of map with concentration location and averages, with model in background
from windtunnel.concentration.utils import stl_to_2d_plot, add_crosses, show_multiple_projections
from windtunnel.concentration.utils import plot_stl_3d, add_crosses_3d
from windtunnel.concentration.utils import add_velocity_field
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Path to your STL file
stl_file = "/home/sabrina/Schreibtisch/Arbeit_2025/FreeCAD/20240206_BfS_model_scale_complete.stl"
    
# Define XY coordinates of concentration measurements

#model scale
"""
points = [
   #(-1020,0),
   (-970,-105),
   (930,270),
   (-850,0),
    (15, 25),
    (20, 10),
    (25, 15),
    (30, 30),
    #(x,y)
]
"""

#Read-in from avg files
from windtunnel.concentration.utils import load_avg_file
file_path = "/home/sabrina/Desktop/Schreibtisch/Arbeit_2025/windtunnel_software/Data/"
file_names = [
    "Point_Data_avg\BFS_BD3_MP01_000_01\_avg_BFS_BD3_MP01_000_01.ts#0",
    "Point_Data_avg\BFS_BD3_MP01_000_01\_avg_BFS_BD3_MP01_000_01.ts#0"
]
points_ms,values_ms, wind_speeds = [],[],[]
for file_name in file_names:
    data_dict = load_avg_file(file_path + file_name)
    x = data_dict['metadata']['x (measurement relativ to source)']['value']
    y = data_dict['metadata']['y (measurement relativ to source)']['value']
    #z.append(data_dict['metadata']['z (measurement relativ to source)']['value'])
    c = data_dict['data'][0]['net_concentration [ppmV]']
    #To test out wind speeds
    wind_speeds.append(data_dict['metadata']['wtref']['value'])
    points_ms.append((x,y))
    values_ms.append(c)
    
ref_position = (0,0)


#####Add velocity field
#Velocity time series values for background overlaying wind field
wind_speeds = [
    0,
    1,
    2,
    3
]
interpolate_windField = True #Interpolate between wind field measurement locations

#Corresponding locations of measured velocites
points_wtref = [
    (-1800,-1800),
    (1800,-1800),
    (-1800,1800),
    (1800,1800)
]


#Average concentration values
values = [9.2484,3.557,72.931, 15, 25, 35, 45]#,c] #[77.262008,
# Define thresholds and corresponding colors
thresholds = [10, 20, 30, 40]
colors = ['blue','green', 'yellow', 'orange', 'red']

#Call stl to polygon print
fig, ax = stl_to_2d_plot(stl_file, projection='xy',toFullScale="False",scaling=1)
#fig, ax = plot_stl_3d(stl_file, azim=0,elev=30,x_range=[-1000,1000],y_range=[-200,200], z_range=[-200,200])
# Add velocity field (speeds only)
add_velocity_field(ax, points_wtref, values=wind_speeds, interpolate=interpolate-windField, grid_density=40, transparencyFactor=0.2,is_latlon=False,toFullScale=False, zorder=5,show_contour_lines=False)
# Add crosses to the plot#
add_crosses(ax, points_ms, values=values_ms, thresholds=thresholds, colors=colors,size=80, linewidth=1.5)
#add_crosses_3d(ax, points, values=values, thresholds=thresholds, colors=colors,size=80, linewidth=1.5)
# Show the plot
ax.set_xlabel("X_ms[mm]")
ax.set_ylabel("Y_ms[mm]")
plt.tight_layout()
plt.savefig("stl_with_crosses.png", dpi=300)
plt.show()

In [ ]:
from windtunnel.concentration.utils import plot_stl_3d, add_crosses_3d, load_data_from_csv
import matplotlib.pyplot as plt
import numpy as np

# Path to your STL file for city structure
stl_file = "/home/sabrina/Schreibtisch/Arbeit_2025/FreeCAD/20240711 UBA1c_2.stl"
#Path to csv file containing locations and concentration values measured in format /x,y,z,c
path_file = "/home/sabrina/Schreibtisch/Arbeit_2025/FreeCAD/UBA/loc+value.csv"
points, values = load_data_from_csv(path_file) #File is in full scale 

# Define 3D points for crosses (x, y, z coordinates)
"""
points = [
    (-970, -105, 50),
    (930, 270, 10),
    (-850, 0, 20),
    (15, 25, 30),
    (20, 10, 15),
    (25, 15, 40),
    (30, 30, 60)
]
# Values for color-coding the crosses
values = [9.2484, 3.557, 72.931, 15, 25, 35, 45]
"""

#For full scale trafo
scale = 400

x_range=None#[-30000,30000]#[-20000,-10000]#[-30000,30000]#[-10000,0]#[-30000,30000]
y_range=None#[-45000,45000]#[5000,15000]#[-45000,45000]
#x_range=[-30000,-10000]#[-30000,30000]#[-20000,-10000]#[-30000,30000]#[-10000,0]#[-30000,30000]
#y_range=[30000,50000]#[-45000,45000]#[5000,15000]#[-45000,45000]
z_range=[-15000,20000]
#To Full scale range
#x_range = [i * scale / 1000  for i in x_range] 
#y_range = [i * scale / 1000  for i in y_range] 
#z_range = [i * scale / 1000  for i in z_range]
fig, ax = plot_stl_3d(stl_file,elev=30,azim=320,x_range=x_range,y_range=y_range,z_range=z_range,toFullScale="True",scaling=scale)
#fig, ax = plot_stl_3d(stl_file,elev=0,azim=0)
# Define thresholds and corresponding colors
thresholds = [10, 20, 30, 40] 
thresholds = [i / 10000 for i in thresholds]
colors = ['blue', 'green', 'yellow', 'orange', 'red']

add_crosses_3d(ax, points, values=values, thresholds=thresholds, colors=colors,
              size=80, linewidth=1.5)
plt.tight_layout()
plt.savefig("stl_with_crosses.png", dpi=300)
plt.show()